In [18]:
import pandas as pd
import sqlite3
from sqlite3 import Error
from IPython.display import display

In [19]:
conn = sqlite3.connect('mimic.db')
cursor = conn.cursor()

Creating tables

In [20]:
cursor.execute("DROP TABLE IF EXISTS Patients")
cursor.execute("DROP TABLE IF EXISTS Admissions")
cursor.execute("DROP TABLE IF EXISTS Diagnoses_icd")
cursor.execute("DROP TABLE IF EXISTS D_icd_diagnoses")
cursor.execute("DROP TABLE IF EXISTS Prescriptions")


sql ='''CREATE TABLE Patients (
    row_id int  NOT NULL,
    subject_id int  NOT NULL,
    gender varchar(5)  NOT NULL,
    dob datetime  NOT NULL,
    dod datetime  NOT NULL,
    dod_hosp datetime  NULL,
    dod_ssn datetime  NULL,
    expire_flag varchar(5)  NOT NULL,
    PRIMARY KEY (subject_id)
  );'''

cursor.execute(sql)


sql ='''CREATE TABLE Admissions (
    row_id int  NOT NULL,
    subject_id int  NOT NULL,
    hadm_id int  NOT NULL,
    admittime datetime  NOT NULL,
    dischtime datetime  NOT NULL,
    deathtime datetime   NULL,
    admission_type varchar(50)  NOT NULL,
    admission_location varchar(50)  NOT NULL,
    discharge_location varchar(50)  NOT NULL,
    insurance varchar(255)  NOT NULL,
    language varchar(10)   NULL,
    religion varchar(50)   NULL,
    marital_status varchar(50)   NULL,
    ethnicity varchar(200)  NOT NULL,
    edregtime datetime   NULL,
    edouttime datetime   NULL,
    diagnosis varchar(300)  NOT NULL,
    hospital_expire_flag int  NOT NULL,
    has_chartevents_data int  NOT NULL,
    PRIMARY KEY (hadm_id),
    FOREIGN KEY (subject_id) REFERENCES Patients(subject_id)
  );'''

cursor.execute(sql)

sql ='''CREATE TABLE D_icd_diagnoses (
    row_id int  NOT NULL,
    icd9_code varchar(10) NOT NULL,
    short_title varchar(50) NOT NULL,
    long_title varchar(300) NOT NULL,
    PRIMARY KEY (icd9_code) 
  );'''
cursor.execute(sql)


sql ='''CREATE TABLE Diagnoses_icd (
    row_id int  NOT NULL,
    subject_id int  NOT NULL,
    hadm_id int  NOT NULL,
     seq_num int NOT NULL,
     icd9_code varchar(10) NOT NULL,
    PRIMARY KEY (hadm_id),
    FOREIGN KEY (icd9_code) REFERENCES D_icd_diagnoses(icd9_code)
  );'''
cursor.execute(sql)


 #insert code to create Prescriptions table
sql ='''CREATE TABLE Prescriptions (
    row_id int  NOT NULL,
    subject_id int  NOT NULL,
    hadm_id int  NOT NULL,
    startdate datetime NULL,
    enddate datetime NULL,
    drug_type varchar(50) NULL,
    drug varchar(300) NULL,
    drug_name_poe varchar(300) NULL,
    drug_name_generic varchar(300) NULL,
    formulary_drug_cd varchar(300) NULL,
    gsn int NULL,
    ndc real NULL,
    prod_strength varchar(300) NULL,
    dose_val_rx varchar(50) NULL,
    dose_unit_rx varchar(50) NULL,
    form_val_disp varchar(50) NULL,
    form_unit_disp varchar(50) NULL,
    route varchar(50) NULL,
    FOREIGN KEY (hadm_id) REFERENCES Diagnoses_icd(hadm_id),
    FOREIGN KEY (subject_id) REFERENCES Patients(subject_id)
  );'''
cursor.execute(sql)



print("Table(s) created successfully........")
conn.commit()

Table(s) created successfully........


In [21]:
patients = pd.read_csv('../../mimic-iii-clinical-database-1.4/PATIENTS.csv.gz', compression='gzip')
patients.to_sql('Patients', conn, if_exists='replace', index = False)
conn.commit()


In [22]:
admissions = pd.read_csv('../../mimic-iii-clinical-database-1.4/ADMISSIONS.csv.gz', compression='gzip')
admissions.to_sql('Admissions', conn, if_exists='replace', index = False)
conn.commit()



In [23]:
D_icd_diagnoses = pd.read_csv('../../mimic-iii-clinical-database-1.4/D_ICD_DIAGNOSES.csv.gz', compression='gzip')
D_icd_diagnoses.to_sql('D_icd_diagnoses', conn, if_exists='replace', index = False)
conn.commit()


In [24]:
Diagnoses_icd = pd.read_csv('../../mimic-iii-clinical-database-1.4/DIAGNOSES_ICD.csv.gz', compression='gzip')
Diagnoses_icd.to_sql('Diagnoses_icd', conn, if_exists='replace', index = False)
conn.commit()


In [25]:
Prescriptions = pd.read_csv('../../mimic-iii-clinical-database-1.4/PRESCRIPTIONS.csv.gz', compression='gzip', low_memory=False)
Prescriptions.to_sql('Prescriptions', conn, if_exists='replace', index = False)
conn.commit()

In [17]:
Prescriptions.head()

,ROW_ID,SUBJECT_ID,HADM_ID,ICUSTAY_ID,STARTDATE,ENDDATE,DRUG_TYPE,DRUG,DRUG_NAME_POE,DRUG_NAME_GENERIC,FORMULARY_DRUG_CD,GSN,NDC,PROD_STRENGTH,DOSE_VAL_RX,DOSE_UNIT_RX,FORM_VAL_DISP,FORM_UNIT_DISP,ROUTE
0,2214776,6,107064,NaN,2175-06-11 00:00:00,2175-06-12 00:00:00,MAIN,Tacrolimus,Tacrolimus,Tacrolimus,TACR1,021796,469061711.0,1mg Capsule,2,mg,2,CAP,PO
1,2214775,6,107064,NaN,2175-06-11 00:00:00,2175-06-12 00:00:00,MAIN,Warfarin,Warfarin,Warfarin,WARF5,006562,56017275.0,5mg Tablet,5,mg,1,TAB,PO
2,2215524,6,107064,NaN,2175-06-11 00:00:00,2175-06-12 00:00:00,MAIN,Heparin Sodium,NaN,NaN,HEPAPREMIX,006522,338055002.0,"25,000 unit Premix Bag","25,000",UNIT,1,BAG,IV
3,2216265,6,107064,NaN,2175-06-11 00:00:00,2175-06-12 00:00:00,BASE,D5W,NaN,NaN,HEPBASE,NaN,0.0,HEPARIN BASE,250,ml,250,ml,IV
4,2214773,6,107064,NaN,2175-06-11 00:00:00,2175-06-12 00:00:00,MAIN,Furosemide,Furosemide,Furosemide,FURO20,008208,54829725.0,20mg Tablet,20,mg,1,TAB,PO


In [ ]:
sql='''SELECT COUNT(*) from patients'''
pd.read_sql(sql, conn)


In [ ]:
print("hello inna")